In [8]:
#Librerías
import pandas as pd
import matplotlib.pyplot as plt
import itertools
import numpy as np
import gc
from datetime import datetime

#Configuración de parametros de Pandas para mejor visualización
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.precision', 3)
pd.set_option('plotting.backend', 'matplotlib') 
pd.options.mode.chained_assignment = None

In [9]:
num_lags_principal=36
num_lags_otros=13

In [10]:
def percentage_safe(numerator: pd.Series, denominator: pd.Series, dtype='float32', fillna=None) -> pd.Series:
    """
    Calcula un porcentaje seguro como numerator / denominator.
    - Reemplaza divisiones por cero o NaN con NaN.
    - Opcionalmente convierte a float32.
    - Puede rellenar NaNs con `fillna`.
    """
    result = (numerator / denominator).mask((denominator == 0) | (denominator.isna()))
    if fillna is not None:
        result = result.fillna(fillna)
    return result.astype(dtype)

In [11]:
def reduce_mem_usage(df):
    """Itera por las columnas del DataFrame y modifica el tipo de datos para reducir uso de memoria."""
    start_mem = df.memory_usage().sum() / 1024**2
    print(f'Uso de memoria inicial del DataFrame: {start_mem:.2f} MB')

    for col in df.columns:
        col_type = df[col].dtype

        if pd.api.types.is_numeric_dtype(col_type):
            c_min = df[col].min()
            c_max = df[col].max()

            if pd.api.types.is_integer_dtype(col_type):
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            # Sólo convertir a categoría si no lo es ya
            if not pd.api.types.is_categorical_dtype(df[col]):
                df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print(f'Uso de memoria final del DataFrame: {end_mem:.2f} MB')
    print(f'Memoria reducida en un {(100 * (start_mem - end_mem) / start_mem):.2f}%')
    return df

In [12]:
def clasificar_estado_producto(df: pd.DataFrame) -> None:
    """
    Clasifica el estado de los productos en un DataFrame basado en su ciclo de vida y ventas.

    La función evalúa una serie de condiciones y crea una nueva columna 'estado_producto'
    en el DataFrame proporcionado. La modificación se hace 'inplace'.

    Args:
        df (pd.DataFrame): El DataFrame que contiene los datos de los productos.
                           Debe incluir las columnas:
                           - 'meses_vida_producto'
                           - 'producto_crecimiento_ventas_suavizado'
                           - 'share_producto_en_categoria_suavizado'
    
    Returns:
        None: La función modifica el DataFrame directamente.
    """
    # Lista de condiciones en el orden de prioridad requerido
    condiciones = [
        (df['meses_vida_producto'] >= 0)& (df['meses_vida_producto'] <= 3),
        (df['producto_crecimiento_ventas_suavizado'] == 0),
        (df['producto_crecimiento_ventas_suavizado'] > 0.1) & (df['share_producto_en_categoria_suavizado'] > 0.1),
        (df['producto_crecimiento_ventas_suavizado'] < -0.1) & (df['share_producto_en_categoria_suavizado'] < -0.1)
    ]

    # Lista de resultados correspondientes a cada condición
    resultados = [
        'inicial',
        'sin ventas',
        'crecimiento',
        'contraccion'
    ]

    # El valor por defecto si ninguna condición se cumple es 'estable'
    df['estado_producto'] = np.select(condiciones, resultados, default='estable')


In [13]:
def calcular_ventanas_moviles(df, campo_base, prefijo_lag, tipo_agregacion, nombre_sufijo=None):
    """
    Calcula estadísticas móviles sobre ventanas específicas (3, 6 y 12 periodos).

    Parámetros:
    - df: DataFrame.
    - campo_base: nombre de la columna base (ej. "cat_total_tn").
    - prefijo_lag: parte de la cadena que compone la columna (ej. "_ma_").
    - tipo_agregacion: 'mean', 'std', 'min', 'max', 'median'.
    - nombre_sufijo: sufijo opcional para el nombre de la columna resultante.
    """
    if tipo_agregacion not in {"mean", "std", "min", "max", "median"}:
        raise ValueError("Agregación no soportada.")

    ventanas = [3, 6, 12]
    for ventana in ventanas:
        # Se genera el nombre completo de la columna que ya existe en el DataFrame.
        columna_ventana = f"{campo_base}{prefijo_lag}{ventana}"
        nombre_col = f"{campo_base}_{tipo_agregacion}_movil_{ventana}"
        if nombre_sufijo:
            nombre_col += f"_{nombre_sufijo}"
        # Aquí se aplica la agregación; en realidad, dado que se opera sobre una sola columna,
        # esto simplemente devolverá el mismo valor (o podría servir para dar formato).
        df[nombre_col] = getattr(df[[columna_ventana]], tipo_agregacion)(axis=1)
    
    return df

# Cargar Datos

In [14]:
#cargar csv
sellin = pd.read_csv('sell-in.txt.gz', sep='\t')
productos = pd.read_csv('tb_productos.txt', sep='\t')
stocks = pd.read_csv('tb_stocks.txt', sep='\t')

# Armado inicial Dataset

In [15]:
#marcar todas las ventas como cliente_activo
sellin['cliente_activo']=1
sellin['producto_activo']=1

In [16]:
max_periodo=sellin['periodo'].max()
max_periodo

201912

In [17]:
vida_producto = sellin[sellin['producto_activo'] == 1].groupby('product_id')['periodo'].agg(
    inicio_vida_p='min', fin_vida_p='max'
).reset_index()

vida_cliente = sellin[sellin['cliente_activo'] == 1].groupby('customer_id')['periodo'].agg(
    inicio_vida_c='min'
).reset_index()

vida_cliente['fin_vida_c'] = max_periodo

In [18]:
#Paso 2: Cruce cliente-producto
cruce_cp = vida_cliente.merge(vida_producto, how='cross')  # Esto da un DataFrame con customer_id y product_id
cruce_cp.head(10)

,customer_id,inicio_vida_c,fin_vida_c,product_id,inicio_vida_p,fin_vida_p
0,10001,201701,201912,20001,201701,201912
1,10001,201701,201912,20002,201701,201912
2,10001,201701,201912,20003,201701,201912
3,10001,201701,201912,20004,201701,201912
4,10001,201701,201912,20005,201701,201912
5,10001,201701,201912,20006,201701,201912
6,10001,201701,201912,20007,201701,201912
7,10001,201701,201912,20008,201701,201912
8,10001,201701,201912,20009,201701,201912
9,10001,201701,201912,20010,201701,201912


In [19]:
#Paso 3: Crear periodos válidos
cruce_cp['inicio_vida'] = cruce_cp[['inicio_vida_c', 'inicio_vida_p']].max(axis=1)
cruce_cp['fin_vida'] = cruce_cp[['fin_vida_c', 'fin_vida_p']].min(axis=1)

# Filtrar combinaciones donde haya al menos un periodo posible
cruce_cp = cruce_cp[cruce_cp['inicio_vida'] <= cruce_cp['fin_vida']]

In [20]:
# Convertir las columnas a datetime primero
inicio_dt = pd.to_datetime(cruce_cp['inicio_vida'].astype(str), format='%Y%m')
fin_dt = pd.to_datetime(cruce_cp['fin_vida'].astype(str), format='%Y%m')

# Generar las listas de periodos como YYYYMM para cada par de fechas
cruce_cp['periodo'] = [
    [int(d.strftime('%Y%m')) for d in pd.date_range(start=ini, end=fin, freq='MS')]
    for ini, fin in zip(inicio_dt, fin_dt)
]

In [21]:
# Asegurarse de tener customer_id, product_id y periodo antes de explotar
combinaciones_validas = cruce_cp[['customer_id', 'product_id', 'periodo']].explode('periodo')

In [22]:
#Paso 5: Merge y completar faltantes
df_completo = combinaciones_validas.merge(sellin, on=['periodo', 'customer_id', 'product_id'], how='left')

In [23]:
#Paso 5: Merge y completar faltantes
df_completo = combinaciones_validas.merge(sellin, on=['periodo', 'customer_id', 'product_id'], how='left')

for col in ['cust_request_qty', 'cust_request_tn', 'tn']:
    df_completo[col] = df_completo[col].fillna(0)

df_completo['cliente_activo'] = df_completo['cliente_activo'].fillna(1)
df_completo['producto_activo'] = df_completo['producto_activo'].fillna(1)

In [24]:
df_completo.shape

(17173448, 9)

In [25]:
df_completo = df_completo.merge(
    cruce_cp[['customer_id', 'product_id', 'inicio_vida_c', 'fin_vida_c', 'inicio_vida_p', 'fin_vida_p', 'inicio_vida', 'fin_vida']],
    on=['customer_id', 'product_id'],
    how='left'
)
df_completo.head(5)

,customer_id,product_id,periodo,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn,cliente_activo,producto_activo,inicio_vida_c,fin_vida_c,inicio_vida_p,fin_vida_p,inicio_vida,fin_vida
0,10001,20001,201701,0.0,11.0,99.439,99.439,1.0,1.0,201701,201912,201701,201912,201701,201912
1,10001,20001,201702,0.0,23.0,198.844,198.844,1.0,1.0,201701,201912,201701,201912,201701,201912
2,10001,20001,201703,0.0,33.0,92.465,92.465,1.0,1.0,201701,201912,201701,201912,201701,201912
3,10001,20001,201704,0.0,8.0,13.297,13.297,1.0,1.0,201701,201912,201701,201912,201701,201912
4,10001,20001,201705,0.0,15.0,101.207,101.006,1.0,1.0,201701,201912,201701,201912,201701,201912


In [26]:
# Convertimos los campos a datetime usando el primer día del mes
df_completo['fecha_periodo'] = pd.to_datetime(df_completo['periodo'].astype(str) + '01', format='%Y%m%d')
df_completo['fecha_inicio_c'] = pd.to_datetime(df_completo['inicio_vida_c'].astype(str) + '01', format='%Y%m%d')
df_completo['fecha_inicio_p'] = pd.to_datetime(df_completo['inicio_vida_p'].astype(str) + '01', format='%Y%m%d')

# Calculamos meses de vida +1 para que el primer mes dé 1
df_completo['meses_vida_cliente'] = (
    (df_completo['fecha_periodo'].dt.year - df_completo['fecha_inicio_c'].dt.year) * 12 +
    (df_completo['fecha_periodo'].dt.month - df_completo['fecha_inicio_c'].dt.month) + 1
)

df_completo['meses_vida_producto'] = (
    (df_completo['fecha_periodo'].dt.year - df_completo['fecha_inicio_p'].dt.year) * 12 +
    (df_completo['fecha_periodo'].dt.month - df_completo['fecha_inicio_p'].dt.month) + 1
)


In [27]:
df_completo.drop(columns=['fin_vida_c','fecha_periodo','inicio_vida','fin_vida','fecha_inicio_c','fecha_inicio_p'], inplace=True)

In [28]:
#Completar precios cuidados

plan_ref = sellin[['periodo', 'product_id', 'plan_precios_cuidados']].drop_duplicates()
plan_ref.head(5)
# Completar plan_precios_cuidados según referencia
df_full = df_completo.drop(columns=['plan_precios_cuidados'], inplace=True)  # eliminamos el incompleto si está
df_full = df_completo.merge(plan_ref, on=['periodo', 'product_id'], how='left')
df_full['plan_precios_cuidados']=df_full['plan_precios_cuidados'].fillna(0).astype(int)

In [29]:
del plan_ref
del combinaciones_validas
del cruce_cp
del df_completo
del sellin
gc.collect()

84

In [30]:
df = df_full.groupby(['periodo', 'product_id']).agg(
    tn=('tn', 'sum'),
    avg_tn=('tn', 'mean'),
    std_tn=('tn', 'std'),
    clientes_distintos=('customer_id', 'nunique'),
    plan_precios_cuidados=('plan_precios_cuidados', 'first'),
    cust_request_qty=('cust_request_qty','sum'),
    cust_request_tn=('cust_request_tn','sum'),
    inicio_vida_p=('inicio_vida_p','min'),
    fin_vida_p=('fin_vida_p','max')
    
    
).reset_index()

In [31]:
#unir en un solo dataset
df = (
    df
    .merge(productos, on='product_id', how='left')
    .merge(stocks, on=['product_id', 'periodo'], how='left')
)

In [32]:
df.drop(columns=['descripcion'], inplace=True)

# Genero los otros df

In [33]:
df_full= (
    df_full
    .merge(productos, on='product_id', how='left')
    .merge(stocks, on=['product_id', 'periodo'], how='left')
)

In [34]:
df_tn_total = (
    df_full.groupby(['periodo'])
      .agg(
          total_total_tn=('tn', 'sum'),
          total_avg_tn=('tn', 'mean'),
          total_std_tn=('tn', 'std'),
          total_min_tn=('tn', 'min'),
          total_max_tn=('tn', 'max'),
          total_productos_distintos=('product_id', 'nunique'),
          total_clientes_distintos=('customer_id', 'nunique')
      )
      .reset_index()
)

In [35]:
df_categoria = (
    df_full.groupby(['periodo', 'cat1', 'cat2', 'cat3'])
      .agg(
          cat_total_tn=('tn', 'sum'),
          cat_avg_tn=('tn', 'mean'),
          cat_std_tn=('tn', 'std'),
          cat_min_tn=('tn', 'min'),
          cat_max_tn=('tn', 'max'),
          cat_productos_distintos=('product_id', 'nunique')
      )
      .reset_index()
)

In [36]:
df_producto = df_full.groupby(['periodo', 'product_id']).agg(
    producto_total_tn=('tn', 'sum'),
    producto_avg_tn=('tn', 'mean'),
    producto_std_tn=('tn', 'std'),
    producto_min_tn=('tn', 'min'),
    producto_max_tn=('tn', 'max'),
    producto_clientes_distintos=('customer_id', 'nunique')
).reset_index()

In [37]:
df_meses_vida = (
    df_full
    .groupby(['periodo', 'product_id'], as_index=True)
    ['meses_vida_producto']
    .first()
    .reset_index()
)

In [38]:
del df_full
gc.collect()
del stocks
gc.collect()

0

In [39]:
# Crear una fecha de inicio del mes
df['fecha'] = pd.to_datetime(df['periodo'], format="%Y%m")

# Obtener cantidad de días del mes original
#df['dias_en_mes'] = df['fecha'].dt.days_in_month

# Crear fechas para 1 y 2 meses posteriores
df['fecha_4'] = df['fecha'] + pd.DateOffset(months=4)
#df['fecha_2'] = df['fecha'] + pd.DateOffset(months=2)

# Obtener días de cada uno de esos meses
df['dias_mes_4'] = df['fecha_4'].dt.days_in_month.astype(int)
#df['dias_mes_2'] = df['fecha_2'].dt.days_in_month

# Agregar año, mes y trimestre
df['anio'] = df['fecha'].dt.year.astype(int)
df['mes'] = df['fecha'].dt.month.astype(int)
df['trimestre'] = df['fecha'].dt.quarter

# Limpiar columnas temporales
df.drop(columns=['fecha', 'fecha_4'], inplace=True)

In [40]:
#Defino mes outlier
df['outlier'] = (df['periodo'] == 201908).astype(int)
df['outlier-2'] = (df['periodo'] == 201906).astype(int)

# LAGS serie principal

In [41]:
# ordenamiento previo:

df.sort_values(by=[ 'product_id', 'periodo'], inplace=True)

df.reset_index(drop=True, inplace=True)

# --- Generación de 36 Lags para la columna 'tn' (dejando NaNs) ---
print("\nGenerando 36 lags para la columna 'tn' (los NaNs serán preservados)...")
for i in range(1, 37): # Genera lags desde 1 hasta 36
    df[f'tn_lag_{i}'] = df.groupby(['product_id'])['tn'].shift(i)

print(f"Se han generado {len(range(1, num_lags_principal+1))} columnas de lag para 'tn'.")


Generando 36 lags para la columna 'tn' (los NaNs serán preservados)...
Se han generado 36 columnas de lag para 'tn'.


In [42]:
# Media móvil de 3 meses: tn + tn_lag_1 + tn_lag_2
df["tn_media_movil_3"] = df[["tn", "tn_lag_1", "tn_lag_2"]].mean(axis=1)

# Media móvil de 6 meses: tn + tn_lag_1 a tn_lag_5
df["tn_media_movil_6"] = df[["tn"] + [f"tn_lag_{i}" for i in range(1, 6)]].mean(axis=1)

# Media móvil de 12 meses: tn + tn_lag_1 a tn_lag_11
df["tn_media_movil_12"] = df[["tn"] + [f"tn_lag_{i}" for i in range(1, 12)]].mean(axis=1)

In [43]:
# Desviación estándar de 3 meses: tn + tn_lag_1 + tn_lag_2
df["tn_std_movil_3"] = df[["tn", "tn_lag_1", "tn_lag_2"]].std(axis=1)

# Desviación estándar de 6 meses: tn + tn_lag_1 a tn_lag_5
df["tn_std_movil_6"] = df[["tn"] + [f"tn_lag_{i}" for i in range(1, 6)]].std(axis=1)

# Desviación estándar de 12 meses: tn + tn_lag_1 a tn_lag_11
df["tn_std_movil_12"] = df[["tn"] + [f"tn_lag_{i}" for i in range(1, 12)]].std(axis=1)

In [44]:
# Mínimo de 3 meses: tn + tn_lag_1 + tn_lag_2
df["tn_min_movil_3"] = df[["tn", "tn_lag_1", "tn_lag_2"]].min(axis=1)

# Mínimo de 6 meses: tn + tn_lag_1 a tn_lag_5
df["tn_min_movil_6"] = df[["tn"] + [f"tn_lag_{i}" for i in range(1, 6)]].min(axis=1)

# Mínimo de 12 meses: tn + tn_lag_1 a tn_lag_11
df["tn_min_movil_12"] = df[["tn"] + [f"tn_lag_{i}" for i in range(1, 12)]].min(axis=1)

In [45]:
#Delta al promedio

df["delta1_media_movil_12"]=df['tn']-df["tn_media_movil_12"]
df["delta1_media_movil_3"]=df['tn']-df["tn_media_movil_3"]
df["delta1_media_movil_6"]=df['tn']-df["tn_media_movil_6"]

In [46]:
# --- Generar las diferencias (delta_tn_X) para todos los lags (dejando NaNs) ---
print("\nGenerando columnas de diferencia (delta_tn_X) (los NaNs serán preservados)...")
for i in range(1, num_lags_principal+1): 
    df[f'delta_tn_{i}'] = df['tn'] - df[f'tn_lag_{i}']


Generando columnas de diferencia (delta_tn_X) (los NaNs serán preservados)...


# serie total

In [47]:
# Asegurate de que esté ordenado por periodo
df_tn_total = df_tn_total.sort_values('periodo').reset_index(drop=True)

# Columnas sobre las que querés generar lags
#columns_to_lag = [
#    'total_total_tn', 
#    'total_avg_tn', 
#    'total_std_tn', 
#    'total_productos_distintos', 
#    'total_clientes_distintos'
#]

# Generar lags del 1 al 12
#for col in columns_to_lag:
#    for lag in range(1, num_lags_otros+1):
#        df_tn_total[f'{col}_lag{lag}'] = df_tn_total[col].shift(lag)


In [48]:
for lag in range(1, num_lags_otros + 1):
    # Creamos la columna de lag con un nombre consistente
    lag_col = f'total_total_tn_lag_{lag}'
    df_tn_total[lag_col] = df_tn_total['total_total_tn'].shift(lag)
    
    # Calculamos la diferencia entre el valor original y el valor en el lag
    df_tn_total[f'total_total_tn_diff_{lag}'] = df_tn_total['total_total_tn'] - df_tn_total[lag_col]

In [49]:
df_tn_total.head()

,periodo,total_total_tn,total_avg_tn,total_std_tn,total_min_tn,total_max_tn,total_productos_distintos,total_clientes_distintos,total_total_tn_lag_1,total_total_tn_diff_1,total_total_tn_lag_2,total_total_tn_diff_2,total_total_tn_lag_3,total_total_tn_diff_3,total_total_tn_lag_4,total_total_tn_diff_4,total_total_tn_lag_5,total_total_tn_diff_5,total_total_tn_lag_6,total_total_tn_diff_6,total_total_tn_lag_7,total_total_tn_diff_7,total_total_tn_lag_8,total_total_tn_diff_8,total_total_tn_lag_9,total_total_tn_diff_9,total_total_tn_lag_10,total_total_tn_diff_10,total_total_tn_lag_11,total_total_tn_diff_11,total_total_tn_lag_12,total_total_tn_diff_12,total_total_tn_lag_13,total_total_tn_diff_13
0,201701,34057.318,0.100,1.274,0.0,184.729,785,433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201702,34568.653,0.091,1.107,0.0,198.844,786,485,34057.318,511.335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201703,46040.596,0.115,1.517,0.0,295.439,793,506,34568.653,11471.943,34057.318,11983.279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201704,39625.524,0.097,1.346,0.0,264.714,797,512,46040.596,-6415.072,34568.653,5056.871,34057.318,5568.207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201705,45579.633,0.109,1.458,0.0,216.361,813,513,39625.524,5954.109,46040.596,-460.963,34568.653,11010.980,34057.318,11522.315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
    
for window in [3, 6, 12]:
    df_tn_total[f'total_total_tn_ma_{window}'] = (
        df_tn_total['total_total_tn']
        .transform(lambda x: x.shift(1).rolling(window=window, min_periods=1).mean())
    )

In [51]:
for window in [3, 6, 12]:
    df_tn_total[f'total_total_tn_min_{window}'] = (
        df_tn_total['total_total_tn']
        .transform(lambda x: x.shift(1).rolling(window=window, min_periods=1).min())
    )

In [52]:
for window in [3, 6, 12]:
    df_tn_total[f'total_total_tn_std_{window}'] = (
        df_tn_total['total_total_tn']
        .transform(lambda x: x.shift(1).rolling(window=window, min_periods=1).std())
    )

In [53]:
df_tn_total = calcular_ventanas_moviles(
    df=df_tn_total,
    campo_base="total_total_tn",
    prefijo_lag="_ma_",
    tipo_agregacion="mean"
)


In [54]:
df_tn_total = calcular_ventanas_moviles(
    df=df_tn_total,
    campo_base="total_total_tn",
    prefijo_lag="_min_",
    tipo_agregacion="min"
)

In [55]:
df_tn_total = calcular_ventanas_moviles(
    df=df_tn_total,
    campo_base="total_total_tn",
    prefijo_lag="_std_",
    tipo_agregacion="std"
)

In [56]:
df_tn_total = reduce_mem_usage(df_tn_total)


Uso de memoria inicial del DataFrame: 0.01 MB
Uso de memoria final del DataFrame: 0.00 MB
Memoria reducida en un 69.58%


In [57]:
df = reduce_mem_usage(df)

Uso de memoria inicial del DataFrame: 24.89 MB
Uso de memoria final del DataFrame: 6.68 MB
Memoria reducida en un 73.17%


C:\Users\agust\AppData\Local\Temp\ipykernel_7208\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):
C:\Users\agust\AppData\Local\Temp\ipykernel_7208\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):
C:\Users\agust\AppData\Local\Temp\ipykernel_7208\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):
C:\Users\agust\AppData\Local\Temp\ipykernel_7208\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.Categorical

In [58]:
gc.collect()

0

In [59]:
df.shape

(31522, 107)

In [60]:
df = df.merge(
    df_tn_total,
    on=['periodo'],
    how='left'
)

In [61]:
del df_tn_total
gc.collect()

0

In [62]:
df = reduce_mem_usage(df)
gc.collect()

Uso de memoria inicial del DataFrame: 10.29 MB


C:\Users\agust\AppData\Local\Temp\ipykernel_7208\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):


Uso de memoria final del DataFrame: 10.29 MB
Memoria reducida en un 0.00%


0

In [63]:
df.shape

(31522, 158)

In [64]:
df = reduce_mem_usage(df)
gc.collect()

Uso de memoria inicial del DataFrame: 10.29 MB
Uso de memoria final del DataFrame: 10.29 MB
Memoria reducida en un 0.00%


C:\Users\agust\AppData\Local\Temp\ipykernel_7208\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):


0

# Clase

In [65]:
#Definir clase
# Ordenar por grupo y tiempo antes del shift negativo
df.sort_values([ 'product_id', 'periodo'], inplace=True)
df.reset_index(drop=True, inplace=True)

# Crear columna con el valor de tn dentro de dos períodos futuros
df['tn_+_2'] = df.groupby(['product_id'])['tn'].shift(-2)

# Calcular diferencia (puede usarse como proxy de tendencia)
df['clase'] = df['tn_+_2'] - df['tn'].fillna(0)

# Limpiar columna temporal
del df['tn_+_2']
df['clase'] = df['clase'].fillna(0)

C:\Users\agust\AppData\Local\Temp\ipykernel_7208\1709805598.py:7: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['tn_+_2'] = df.groupby(['product_id'])['tn'].shift(-2)
C:\Users\agust\AppData\Local\Temp\ipykernel_7208\1709805598.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['clase'] = df['tn_+_2'] - df['tn'].fillna(0)


In [66]:
df = reduce_mem_usage(df)
gc.collect()

Uso de memoria inicial del DataFrame: 10.35 MB


C:\Users\agust\AppData\Local\Temp\ipykernel_7208\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):


Uso de memoria final del DataFrame: 10.35 MB
Memoria reducida en un 0.00%


0

In [67]:
gc.collect()

0

# Serie Categoria

In [68]:
for lag in range(1, num_lags_otros + 1):
    df_categoria[f'cat_total_tn_lag_{lag}'] = (
        df_categoria.groupby(['cat1', 'cat2', 'cat3'])['cat_total_tn'].shift(lag)
    )
    df_categoria[f'cat_delta_tn_lag_{lag}'] = (
        df_categoria['cat_total_tn'] - df_categoria[f'cat_total_tn_lag_{lag}']
    )


In [69]:
for window in [3, 6, 12]:
    df_categoria[f'cat_total_tn_ma_{window}'] = (
        df_categoria
        .groupby(['cat1', 'cat2', 'cat3'])['cat_total_tn']
        .transform(lambda x: x.shift(1).rolling(window=window, min_periods=1).mean())
    )

In [70]:
for window in [3, 6, 12]:
    df_categoria[f'cat_total_tn_min_{window}'] = (
        df_categoria
        .groupby(['cat1', 'cat2', 'cat3'])['cat_total_tn']
        .transform(lambda x: x.shift(1).rolling(window=window, min_periods=1).min())
    )

In [71]:
for window in [3, 6, 12]:
    df_categoria[f'cat_total_tn_std_{window}'] = (
        df_categoria
        .groupby(['cat1', 'cat2', 'cat3'])['cat_total_tn']
        .transform(lambda x: x.shift(1).rolling(window=window, min_periods=1).std())
    )

In [72]:
df_categoria.head()

,periodo,cat1,cat2,cat3,cat_total_tn,cat_avg_tn,cat_std_tn,cat_min_tn,cat_max_tn,cat_productos_distintos,cat_total_tn_lag_1,cat_delta_tn_lag_1,cat_total_tn_lag_2,cat_delta_tn_lag_2,cat_total_tn_lag_3,cat_delta_tn_lag_3,cat_total_tn_lag_4,cat_delta_tn_lag_4,cat_total_tn_lag_5,cat_delta_tn_lag_5,cat_total_tn_lag_6,cat_delta_tn_lag_6,cat_total_tn_lag_7,cat_delta_tn_lag_7,cat_total_tn_lag_8,cat_delta_tn_lag_8,cat_total_tn_lag_9,cat_delta_tn_lag_9,cat_total_tn_lag_10,cat_delta_tn_lag_10,cat_total_tn_lag_11,cat_delta_tn_lag_11,cat_total_tn_lag_12,cat_delta_tn_lag_12,cat_total_tn_lag_13,cat_delta_tn_lag_13,cat_total_tn_ma_3,cat_total_tn_ma_6,cat_total_tn_ma_12,cat_total_tn_min_3,cat_total_tn_min_6,cat_total_tn_min_12,cat_total_tn_std_3,cat_total_tn_std_6,cat_total_tn_std_12
0,201701,FOODS,ADEREZOS,Aji Picante,6.321,0.015,0.070,0.0,0.781,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201701,FOODS,ADEREZOS,Barbacoa,11.820,0.027,0.178,0.0,2.306,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201701,FOODS,ADEREZOS,Chimichurri,6.032,0.014,0.058,0.0,0.570,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201701,FOODS,ADEREZOS,Ketchup,393.767,0.130,1.093,0.0,26.601,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201701,FOODS,ADEREZOS,Mayonesa,3742.878,0.279,3.165,0.0,156.651,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
df_categoria = calcular_ventanas_moviles(
    df=df_categoria,
    campo_base="cat_total_tn",
    prefijo_lag="_ma_",
    tipo_agregacion="mean"
)


In [74]:
df_categoria = calcular_ventanas_moviles(
    df=df_categoria,
    campo_base="cat_total_tn",
    prefijo_lag="_min_",
    tipo_agregacion="min"
)

In [75]:
df_categoria = calcular_ventanas_moviles(
    df=df_categoria,
    campo_base="cat_total_tn",
    prefijo_lag="_std_",
    tipo_agregacion="std"
)

In [76]:
df_categoria = reduce_mem_usage(df_categoria)
gc.collect()

Uso de memoria inicial del DataFrame: 1.28 MB
Uso de memoria final del DataFrame: 0.37 MB
Memoria reducida en un 71.01%


C:\Users\agust\AppData\Local\Temp\ipykernel_7208\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):
C:\Users\agust\AppData\Local\Temp\ipykernel_7208\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):
C:\Users\agust\AppData\Local\Temp\ipykernel_7208\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):


0

In [77]:
df_categoria.head(5)

,periodo,cat1,cat2,cat3,cat_total_tn,cat_avg_tn,cat_std_tn,cat_min_tn,cat_max_tn,cat_productos_distintos,cat_total_tn_lag_1,cat_delta_tn_lag_1,cat_total_tn_lag_2,cat_delta_tn_lag_2,cat_total_tn_lag_3,cat_delta_tn_lag_3,cat_total_tn_lag_4,cat_delta_tn_lag_4,cat_total_tn_lag_5,cat_delta_tn_lag_5,cat_total_tn_lag_6,cat_delta_tn_lag_6,cat_total_tn_lag_7,cat_delta_tn_lag_7,cat_total_tn_lag_8,cat_delta_tn_lag_8,cat_total_tn_lag_9,cat_delta_tn_lag_9,cat_total_tn_lag_10,cat_delta_tn_lag_10,cat_total_tn_lag_11,cat_delta_tn_lag_11,cat_total_tn_lag_12,cat_delta_tn_lag_12,cat_total_tn_lag_13,cat_delta_tn_lag_13,cat_total_tn_ma_3,cat_total_tn_ma_6,cat_total_tn_ma_12,cat_total_tn_min_3,cat_total_tn_min_6,cat_total_tn_min_12,cat_total_tn_std_3,cat_total_tn_std_6,cat_total_tn_std_12,cat_total_tn_mean_movil_3,cat_total_tn_mean_movil_6,cat_total_tn_mean_movil_12,cat_total_tn_min_movil_3,cat_total_tn_min_movil_6,cat_total_tn_min_movil_12,cat_total_tn_std_movil_3,cat_total_tn_std_movil_6,cat_total_tn_std_movil_12
0,201701,FOODS,ADEREZOS,Aji Picante,6.320,0.015,0.070,0.0,0.781,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201701,FOODS,ADEREZOS,Barbacoa,11.820,0.027,0.178,0.0,2.307,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201701,FOODS,ADEREZOS,Chimichurri,6.031,0.014,0.058,0.0,0.570,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201701,FOODS,ADEREZOS,Ketchup,393.750,0.130,1.094,0.0,26.594,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201701,FOODS,ADEREZOS,Mayonesa,3742.000,0.279,3.164,0.0,156.625,31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
df_categoria.shape

(3114, 54)

# Serie por producto

In [79]:

df = df.rename(columns={'tn': 'producto_total_tn'})


In [80]:
df_producto = df_producto.sort_values(['product_id', 'periodo'])

for lag in range(1, num_lags_otros+1):
    df_producto[f'producto_total_tn_lag_{lag}'] = (
        df_producto.groupby('product_id')['producto_total_tn'].shift(lag)
    )

In [81]:
# Media móvil de 3 meses: tn + tn_lag_1 + tn_lag_2
df_producto["producto_tn_media_movil_3(con_mes_en_curso)"] = df_producto[["producto_total_tn", "producto_total_tn_lag_1", "producto_total_tn_lag_2"]].mean(axis=1)
df_producto["producto_tn_media_movil_3anteriores"] = df_producto[["producto_total_tn_lag_3", "producto_total_tn_lag_4","producto_total_tn_lag_5",]].mean(axis=1)

df_producto['producto_crecimiento_ventas_suavizado']= percentage_safe(df_producto["producto_tn_media_movil_3(con_mes_en_curso)"] , df_producto["producto_tn_media_movil_3anteriores"])-1

In [82]:
df_producto['producto_clientes_distintos_lag_1'] = (
    df_producto
    .groupby('product_id')['producto_clientes_distintos']
    .shift(1)
)

In [83]:
df_producto['producto_clientes_distintos_growth_1'] = np.where(
    df_producto['producto_clientes_distintos_lag_1'] == 0,
    np.nan,
    (df_producto['producto_clientes_distintos'] - df_producto['producto_clientes_distintos_lag_1']) /
    df_producto['producto_clientes_distintos_lag_1']
)

In [84]:
df_producto = reduce_mem_usage(df_producto)
gc.collect()

Uso de memoria inicial del DataFrame: 6.37 MB
Uso de memoria final del DataFrame: 1.86 MB
Memoria reducida en un 70.75%


0

In [85]:
df_producto.head(5)

,periodo,product_id,producto_total_tn,producto_avg_tn,producto_std_tn,producto_min_tn,producto_max_tn,producto_clientes_distintos,producto_total_tn_lag_1,producto_total_tn_lag_2,producto_total_tn_lag_3,producto_total_tn_lag_4,producto_total_tn_lag_5,producto_total_tn_lag_6,producto_total_tn_lag_7,producto_total_tn_lag_8,producto_total_tn_lag_9,producto_total_tn_lag_10,producto_total_tn_lag_11,producto_total_tn_lag_12,producto_total_tn_lag_13,producto_tn_media_movil_3(con_mes_en_curso),producto_tn_media_movil_3anteriores,producto_crecimiento_ventas_suavizado,producto_clientes_distintos_lag_1,producto_clientes_distintos_growth_1
0,201701,20001,935.0,2.158,13.508,0.0,184.750,433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,935.0,NaN,NaN,NaN,NaN
785,201702,20001,798.0,1.646,11.492,0.0,198.875,485,935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,866.5,NaN,NaN,433.0,0.120
1571,201703,20001,1303.0,2.576,18.500,0.0,295.500,506,798.0,935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1012.0,NaN,NaN,485.0,0.043
2364,201704,20001,1070.0,2.090,17.906,0.0,264.750,512,1303.0,798.0,935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1057.0,935.0,0.131,506.0,0.012
3161,201705,20001,1502.0,2.928,16.906,0.0,216.375,513,1070.0,1303.0,798.0,935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1292.0,866.5,0.491,512.0,0.002


# Unir los productos y categorias dataset

In [86]:
df_producto = (
    df_producto
    .merge(productos, on=['product_id'], how='left')
)

In [87]:
df_producto = (
    df_producto
    .merge(df_categoria, on=['periodo', 'cat1', 'cat2', 'cat3'], how='left')
)

In [88]:
df_producto = reduce_mem_usage(df_producto)
gc.collect()

del df_categoria


df_producto.head(5)

Uso de memoria inicial del DataFrame: 6.79 MB
Uso de memoria final del DataFrame: 5.44 MB
Memoria reducida en un 19.98%


C:\Users\agust\AppData\Local\Temp\ipykernel_7208\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):
C:\Users\agust\AppData\Local\Temp\ipykernel_7208\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):
C:\Users\agust\AppData\Local\Temp\ipykernel_7208\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):
C:\Users\agust\AppData\Local\Temp\ipykernel_7208\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.Categorical

,periodo,product_id,producto_total_tn,producto_avg_tn,producto_std_tn,producto_min_tn,producto_max_tn,producto_clientes_distintos,producto_total_tn_lag_1,producto_total_tn_lag_2,producto_total_tn_lag_3,producto_total_tn_lag_4,producto_total_tn_lag_5,producto_total_tn_lag_6,producto_total_tn_lag_7,producto_total_tn_lag_8,producto_total_tn_lag_9,producto_total_tn_lag_10,producto_total_tn_lag_11,producto_total_tn_lag_12,producto_total_tn_lag_13,producto_tn_media_movil_3(con_mes_en_curso),producto_tn_media_movil_3anteriores,producto_crecimiento_ventas_suavizado,producto_clientes_distintos_lag_1,producto_clientes_distintos_growth_1,cat1,cat2,cat3,brand,sku_size,descripcion,cat_total_tn,cat_avg_tn,cat_std_tn,cat_min_tn,cat_max_tn,cat_productos_distintos,cat_total_tn_lag_1,cat_delta_tn_lag_1,cat_total_tn_lag_2,cat_delta_tn_lag_2,cat_total_tn_lag_3,cat_delta_tn_lag_3,cat_total_tn_lag_4,cat_delta_tn_lag_4,cat_total_tn_lag_5,cat_delta_tn_lag_5,cat_total_tn_lag_6,cat_delta_tn_lag_6,cat_total_tn_lag_7,cat_delta_tn_lag_7,cat_total_tn_lag_8,cat_delta_tn_lag_8,cat_total_tn_lag_9,cat_delta_tn_lag_9,cat_total_tn_lag_10,cat_delta_tn_lag_10,cat_total_tn_lag_11,cat_delta_tn_lag_11,cat_total_tn_lag_12,cat_delta_tn_lag_12,cat_total_tn_lag_13,cat_delta_tn_lag_13,cat_total_tn_ma_3,cat_total_tn_ma_6,cat_total_tn_ma_12,cat_total_tn_min_3,cat_total_tn_min_6,cat_total_tn_min_12,cat_total_tn_std_3,cat_total_tn_std_6,cat_total_tn_std_12,cat_total_tn_mean_movil_3,cat_total_tn_mean_movil_6,cat_total_tn_mean_movil_12,cat_total_tn_min_movil_3,cat_total_tn_min_movil_6,cat_total_tn_min_movil_12,cat_total_tn_std_movil_3,cat_total_tn_std_movil_6,cat_total_tn_std_movil_12
0,201701,20001,935.0,2.158,13.508,0.0,184.750,433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,935.0,NaN,NaN,NaN,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma,3852.0,0.307,3.357,0.0,184.750,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201702,20001,798.0,1.646,11.492,0.0,198.875,485,935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,866.5,NaN,NaN,433.0,0.120,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma,3890.0,0.277,3.021,0.0,198.875,29.0,3852.0,38.594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3852.0,3852.0,3852.0,3852.0,3852.0,3852.0,NaN,NaN,NaN,3852.0,3852.0,3852.0,3852.0,3852.0,3852.0,NaN,NaN,NaN
2,201703,20001,1303.0,2.576,18.500,0.0,295.500,506,798.0,935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1012.0,NaN,NaN,485.0,0.043,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma,5372.0,0.366,4.566,0.0,295.500,29.0,3890.0,1482.000,3852.0,1521.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3870.0,3870.0,3870.0,3852.0,3852.0,3852.0,27.297,27.297,27.297,3870.0,3870.0,3870.0,3852.0,3852.0,3852.0,NaN,NaN,NaN
3,201704,20001,1070.0,2.090,17.906,0.0,264.750,512,1303.0,798.0,935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1057.0,935.0,0.131,506.0,0.012,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma,4220.0,0.275,3.783,0.0,264.750,30.0,5372.0,-1151.000,3890.0,331.500,3852.0,370.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4372.0,4372.0,4372.0,3852.0,3852.0,3852.0,867.500,867.500,867.500,4372.0,4372.0,4372.0,3852.0,3852.0,3852.0,NaN,NaN,NaN
4,201705,20001,1502.0,2.928,16.906,0.0,216.375,513,1070.0,1303.0,798.0,935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1292.0,866.5,0.491,512.0,0.002,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,genoma,5492.0,0.357,4.012,0.0,216.375,30.0,4220.0,1270.000,5372.0,118.812,3890.0,1601.0,3852.0,1640.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4496.0,4336.0,4336.0,3890.0,3852.0,3852.0,778.000,712.000,712.000,4496.0,4336.0,4336.0,3890.0,3852.0,3852.0,NaN,NaN,NaN


In [89]:
df_producto.drop(columns=['descripcion'], inplace=True)

In [90]:
df_producto.head(5)

,periodo,product_id,producto_total_tn,producto_avg_tn,producto_std_tn,producto_min_tn,producto_max_tn,producto_clientes_distintos,producto_total_tn_lag_1,producto_total_tn_lag_2,producto_total_tn_lag_3,producto_total_tn_lag_4,producto_total_tn_lag_5,producto_total_tn_lag_6,producto_total_tn_lag_7,producto_total_tn_lag_8,producto_total_tn_lag_9,producto_total_tn_lag_10,producto_total_tn_lag_11,producto_total_tn_lag_12,producto_total_tn_lag_13,producto_tn_media_movil_3(con_mes_en_curso),producto_tn_media_movil_3anteriores,producto_crecimiento_ventas_suavizado,producto_clientes_distintos_lag_1,producto_clientes_distintos_growth_1,cat1,cat2,cat3,brand,sku_size,cat_total_tn,cat_avg_tn,cat_std_tn,cat_min_tn,cat_max_tn,cat_productos_distintos,cat_total_tn_lag_1,cat_delta_tn_lag_1,cat_total_tn_lag_2,cat_delta_tn_lag_2,cat_total_tn_lag_3,cat_delta_tn_lag_3,cat_total_tn_lag_4,cat_delta_tn_lag_4,cat_total_tn_lag_5,cat_delta_tn_lag_5,cat_total_tn_lag_6,cat_delta_tn_lag_6,cat_total_tn_lag_7,cat_delta_tn_lag_7,cat_total_tn_lag_8,cat_delta_tn_lag_8,cat_total_tn_lag_9,cat_delta_tn_lag_9,cat_total_tn_lag_10,cat_delta_tn_lag_10,cat_total_tn_lag_11,cat_delta_tn_lag_11,cat_total_tn_lag_12,cat_delta_tn_lag_12,cat_total_tn_lag_13,cat_delta_tn_lag_13,cat_total_tn_ma_3,cat_total_tn_ma_6,cat_total_tn_ma_12,cat_total_tn_min_3,cat_total_tn_min_6,cat_total_tn_min_12,cat_total_tn_std_3,cat_total_tn_std_6,cat_total_tn_std_12,cat_total_tn_mean_movil_3,cat_total_tn_mean_movil_6,cat_total_tn_mean_movil_12,cat_total_tn_min_movil_3,cat_total_tn_min_movil_6,cat_total_tn_min_movil_12,cat_total_tn_std_movil_3,cat_total_tn_std_movil_6,cat_total_tn_std_movil_12
0,201701,20001,935.0,2.158,13.508,0.0,184.750,433,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,935.0,NaN,NaN,NaN,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,3852.0,0.307,3.357,0.0,184.750,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201702,20001,798.0,1.646,11.492,0.0,198.875,485,935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,866.5,NaN,NaN,433.0,0.120,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,3890.0,0.277,3.021,0.0,198.875,29.0,3852.0,38.594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3852.0,3852.0,3852.0,3852.0,3852.0,3852.0,NaN,NaN,NaN,3852.0,3852.0,3852.0,3852.0,3852.0,3852.0,NaN,NaN,NaN
2,201703,20001,1303.0,2.576,18.500,0.0,295.500,506,798.0,935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1012.0,NaN,NaN,485.0,0.043,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,5372.0,0.366,4.566,0.0,295.500,29.0,3890.0,1482.000,3852.0,1521.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3870.0,3870.0,3870.0,3852.0,3852.0,3852.0,27.297,27.297,27.297,3870.0,3870.0,3870.0,3852.0,3852.0,3852.0,NaN,NaN,NaN
3,201704,20001,1070.0,2.090,17.906,0.0,264.750,512,1303.0,798.0,935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1057.0,935.0,0.131,506.0,0.012,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,4220.0,0.275,3.783,0.0,264.750,30.0,5372.0,-1151.000,3890.0,331.500,3852.0,370.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4372.0,4372.0,4372.0,3852.0,3852.0,3852.0,867.500,867.500,867.500,4372.0,4372.0,4372.0,3852.0,3852.0,3852.0,NaN,NaN,NaN
4,201705,20001,1502.0,2.928,16.906,0.0,216.375,513,1070.0,1303.0,798.0,935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1292.0,866.5,0.491,512.0,0.002,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,5492.0,0.357,4.012,0.0,216.375,30.0,4220.0,1270.000,5372.0,118.812,3890.0,1601.0,3852.0,1640.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4496.0,4336.0,4336.0,3890.0,3852.0,3852.0,778.000,712.000,712.000,4496.0,4336.0,4336.0,3890.0,3852.0,3852.0,NaN,NaN,NaN


In [91]:
#Penetracion/share producto en la categoria

df_producto['share_producto_en_categoria']=percentage_safe(df_producto['producto_total_tn'],df_producto['cat_total_tn'])

for lag in range(1, num_lags_otros+1):
    df_producto[f"share_producto_en_categoria_lag_{lag}"] = percentage_safe(
        df_producto[f"producto_total_tn_lag_{lag}"],df_producto[f"cat_total_tn_lag_{lag}"]
    )

In [92]:
#tasa de crecimiento share producto en la categoria
df_producto['tasa_crecimiento_share_producto_en_categoria'] = percentage_safe((df_producto['share_producto_en_categoria']-df_producto['share_producto_en_categoria_lag_1']) , 
                                                                              df_producto['share_producto_en_categoria_lag_1'])

for lag in range(1, num_lags_otros+1):
    df_producto[f"tasa_crecimiento_share_producto_en_categoria_lag_{lag}"] = percentage_safe(
        (df_producto[f"share_producto_en_categoria_lag_{lag}"] -
        df_producto[f"share_producto_en_categoria_lag_{lag}"]),
        df_producto[f"share_producto_en_categoria_lag_{lag}"]
    )


C:\Users\agust\AppData\Local\Temp\ipykernel_7208\1936691053.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_producto[f"tasa_crecimiento_share_producto_en_categoria_lag_{lag}"] = percentage_safe(
C:\Users\agust\AppData\Local\Temp\ipykernel_7208\1936691053.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_producto[f"tasa_crecimiento_share_producto_en_categoria_lag_{lag}"] = percentage_safe(
C:\Users\agust\AppData\Local\Temp\ipykernel_7208\1936691053.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usu

In [93]:
# Media móvil de 3 meses: tn + tn_lag_1 + tn_lag_2
# Media móvil de 3 meses: tn + tn_lag_1 + tn_lag_2
df_producto["share_producto_en_categoria_movil_3(con_mes_en_curso)"] = df_producto[["share_producto_en_categoria", "share_producto_en_categoria_lag_1", "share_producto_en_categoria_lag_2"]].mean(axis=1)
df_producto["share_producto_en_categoria_3anteriores"] = df_producto[["share_producto_en_categoria_lag_3", "share_producto_en_categoria_lag_4","share_producto_en_categoria_lag_5",]].mean(axis=1)

df_producto['share_producto_en_categoria_suavizado']= percentage_safe(df_producto["share_producto_en_categoria_movil_3(con_mes_en_curso)"] , df_producto["share_producto_en_categoria_3anteriores"])-1

C:\Users\agust\AppData\Local\Temp\ipykernel_7208\54131255.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_producto["share_producto_en_categoria_movil_3(con_mes_en_curso)"] = df_producto[["share_producto_en_categoria", "share_producto_en_categoria_lag_1", "share_producto_en_categoria_lag_2"]].mean(axis=1)
C:\Users\agust\AppData\Local\Temp\ipykernel_7208\54131255.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_producto["share_producto_en_categoria_3anteriores"] = df_producto[["share_producto_en_categoria_lag_3", "sha

In [94]:
df_producto.drop(columns=['producto_total_tn_lag_1','producto_total_tn_lag_2','producto_total_tn_lag_3','producto_total_tn_lag_4','producto_total_tn_lag_5','producto_total_tn_lag_6','producto_total_tn_lag_7','producto_total_tn_lag_8','producto_total_tn_lag_9','producto_total_tn_lag_10','producto_total_tn_lag_11','producto_total_tn_lag_12','producto_total_tn_lag_13'], inplace=True)

In [95]:
df_producto.shape

(31522, 99)

In [96]:
df_producto = reduce_mem_usage(df_producto)
gc.collect()

C:\Users\agust\AppData\Local\Temp\ipykernel_7208\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):


Uso de memoria inicial del DataFrame: 8.30 MB
Uso de memoria final del DataFrame: 6.44 MB
Memoria reducida en un 22.45%


0

In [97]:
df_producto = (
    df_producto
    .merge(df_meses_vida, on=['periodo', 'product_id'], how='left')
)

In [98]:
del df_meses_vida
gc.collect()

0

In [99]:
def first_not_null(s):
    return s.dropna().iloc[0] if not s.dropna().empty else np.nan

df_ciclo_vida = (
    df_producto
    .groupby(['periodo', 'product_id'], as_index=False)
    .agg({
        'meses_vida_producto': first_not_null,
        'producto_crecimiento_ventas_suavizado': first_not_null,
        'share_producto_en_categoria_suavizado': first_not_null,
    })
)

In [100]:
# 2. Aplicamos la función al DataFrame
clasificar_estado_producto(df_ciclo_vida)


df_ciclo_vida.tail(5)

,periodo,product_id,meses_vida_producto,producto_crecimiento_ventas_suavizado,share_producto_en_categoria_suavizado,estado_producto
31517,201912,21265,10,2.600,2.133,crecimiento
31518,201912,21266,10,1.311,1.189,crecimiento
31519,201912,21267,10,0.147,0.253,crecimiento
31520,201912,21271,30,-0.323,-0.185,contraccion
31521,201912,21276,10,0.888,1.286,crecimiento


In [101]:
df_producto = df_producto.merge(
    df_ciclo_vida[['periodo', 'product_id', 'estado_producto']],
    on=['periodo', 'product_id'],
    how='left'
)

In [102]:
df_producto.shape

(31522, 101)

# Otros productos

In [103]:
df_producto['otros_total_tn_'] = df_producto['cat_total_tn'] - df_producto['producto_total_tn'] 

# Paso 2: ordenar
df_producto = df_producto.sort_values(['product_id', 'periodo'])

for l in range(1, num_lags_otros+1):
    df_producto[f'otros_total_tn_lag{l}'] = (
        df_producto
        .groupby('product_id')['otros_total_tn_']
        .shift(l)
    )



In [104]:
df_producto['otros_avg'] = percentage_safe(df_producto['otros_total_tn_'],(df_producto['cat_productos_distintos']- 1))

for l in range(1, num_lags_otros+1):
    df_producto[f'otros_avg_lag{l}'] = (
        df_producto
        .groupby('product_id')['otros_avg']
        .shift(l)
    )


In [105]:
df_producto.head(8)

,periodo,product_id,producto_total_tn,producto_avg_tn,producto_std_tn,producto_min_tn,producto_max_tn,producto_clientes_distintos,producto_tn_media_movil_3(con_mes_en_curso),producto_tn_media_movil_3anteriores,producto_crecimiento_ventas_suavizado,producto_clientes_distintos_lag_1,producto_clientes_distintos_growth_1,cat1,cat2,cat3,brand,sku_size,cat_total_tn,cat_avg_tn,cat_std_tn,cat_min_tn,cat_max_tn,cat_productos_distintos,cat_total_tn_lag_1,cat_delta_tn_lag_1,cat_total_tn_lag_2,cat_delta_tn_lag_2,cat_total_tn_lag_3,cat_delta_tn_lag_3,cat_total_tn_lag_4,cat_delta_tn_lag_4,cat_total_tn_lag_5,cat_delta_tn_lag_5,cat_total_tn_lag_6,cat_delta_tn_lag_6,cat_total_tn_lag_7,cat_delta_tn_lag_7,cat_total_tn_lag_8,cat_delta_tn_lag_8,cat_total_tn_lag_9,cat_delta_tn_lag_9,cat_total_tn_lag_10,cat_delta_tn_lag_10,cat_total_tn_lag_11,cat_delta_tn_lag_11,cat_total_tn_lag_12,cat_delta_tn_lag_12,cat_total_tn_lag_13,cat_delta_tn_lag_13,cat_total_tn_ma_3,cat_total_tn_ma_6,cat_total_tn_ma_12,cat_total_tn_min_3,cat_total_tn_min_6,cat_total_tn_min_12,cat_total_tn_std_3,cat_total_tn_std_6,cat_total_tn_std_12,cat_total_tn_mean_movil_3,cat_total_tn_mean_movil_6,cat_total_tn_mean_movil_12,cat_total_tn_min_movil_3,cat_total_tn_min_movil_6,cat_total_tn_min_movil_12,cat_total_tn_std_movil_3,cat_total_tn_std_movil_6,cat_total_tn_std_movil_12,share_producto_en_categoria,share_producto_en_categoria_lag_1,share_producto_en_categoria_lag_2,share_producto_en_categoria_lag_3,share_producto_en_categoria_lag_4,share_producto_en_categoria_lag_5,share_producto_en_categoria_lag_6,share_producto_en_categoria_lag_7,share_producto_en_categoria_lag_8,share_producto_en_categoria_lag_9,share_producto_en_categoria_lag_10,share_producto_en_categoria_lag_11,share_producto_en_categoria_lag_12,share_producto_en_categoria_lag_13,tasa_crecimiento_share_producto_en_categoria,tasa_crecimiento_share_producto_en_categoria_lag_1,tasa_crecimiento_share_producto_en_categoria_lag_2,tasa_crecimiento_share_producto_en_categoria_lag_3,tasa_crecimiento_share_producto_en_categoria_lag_4,tasa_crecimiento_share_producto_en_categoria_lag_5,tasa_crecimiento_share_producto_en_categoria_lag_6,tasa_crecimiento_share_producto_en_categoria_lag_7,tasa_crecimiento_share_producto_en_categoria_lag_8,tasa_crecimiento_share_producto_en_categoria_lag_9,tasa_crecimiento_share_producto_en_categoria_lag_10,tasa_crecimiento_share_producto_en_categoria_lag_11,tasa_crecimiento_share_producto_en_categoria_lag_12,tasa_crecimiento_share_producto_en_categoria_lag_13,share_producto_en_categoria_movil_3(con_mes_en_curso),share_producto_en_categoria_3anteriores,share_producto_en_categoria_suavizado,meses_vida_producto,estado_producto,otros_total_tn_,otros_total_tn_lag1,otros_total_tn_lag2,otros_total_tn_lag3,otros_total_tn_lag4,otros_total_tn_lag5,otros_total_tn_lag6,otros_total_tn_lag7,otros_total_tn_lag8,otros_total_tn_lag9,otros_total_tn_lag10,otros_total_tn_lag11,otros_total_tn_lag12,otros_total_tn_lag13,otros_avg,otros_avg_lag1,otros_avg_lag2,otros_avg_lag3,otros_avg_lag4,otros_avg_lag5,otros_avg_lag6,otros_avg_lag7,otros_avg_lag8,otros_avg_lag9,otros_avg_lag10,otros_avg_lag11,otros_avg_lag12,otros_avg_lag13
0,201701,20001,935.0,2.158,13.508,0.0,184.750,433,935.0,NaN,NaN,NaN,NaN,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,3852.0,0.307,3.357,0.0,184.750,29.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.243,NaN,NaN,1,inicial,2916.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201702,20001,798.0,1.646,11.492,0.0,198.875,485,866.5,NaN,NaN,433.0,0.120,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,3890.0,0.277,3.021,0.0,198.875,29.0,3852.0,38.594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [106]:
del df_producto['cat1']
del df_producto['cat2']
del df_producto['cat3']
del df_producto['brand']
del df_producto['sku_size']
gc.collect()

0

# Unir los datasets

In [107]:
df = df.merge(
    df_producto,
    on=['periodo', 'product_id'],
    how='left'
)

In [108]:
df.shape

(31522, 281)

In [109]:
del df_producto
gc.collect()

0

# Exportar 

In [110]:
df['periodo'] = df['periodo'].astype(str).str.replace('-', '').astype(int)

In [111]:

df = df.rename(columns={'producto_total_tn_x': 'tn'})


In [112]:
del df['producto_total_tn_y']


In [113]:
df = reduce_mem_usage(df)
gc.collect()

Uso de memoria inicial del DataFrame: 19.24 MB


C:\Users\agust\AppData\Local\Temp\ipykernel_7208\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):


Uso de memoria final del DataFrame: 18.10 MB
Memoria reducida en un 5.93%


C:\Users\agust\AppData\Local\Temp\ipykernel_7208\3492438140.py:31: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if not pd.api.types.is_categorical_dtype(df[col]):


0

In [114]:
df['cust_request_qty']=df['cust_request_qty'].astype('Int64')
#df['cust_request_qty']=df['cust_request_qty'].astype('Int64')



In [118]:
df.to_parquet('completo_producto.parquet', index=False)

In [115]:
#df_train = df[~df['periodo'].isin([201911, 201912])]
df_train = df.query("periodo != 201911 and periodo != 201912")

df_train.to_parquet('train_producto.parquet', index=False)
print(f"DataFrame de entrenamiento guardado en 'train.parquet' con {len(df_train)} filas.")

# --- 3. Preparar y guardar el DataFrame de predicción en Parquet ---
# Seleccionamos los periodos 201911 y 201912 para el conjunto de predicción.
# Eliminamos la columna 'clase' ya que no será necesaria para la predicción.
# Finalmente, guardamos este DataFrame en un archivo Parquet.
df_predecir = df[df['periodo'].isin([201911, 201912])].copy() # Usar .copy() para evitar SettingWithCopyWarning
df_predecir.drop(columns=['clase'], inplace=True)
df_predecir.to_parquet('predecir_producto.parquet', index=False)
print(f"DataFrame para predicción guardado en 'predecir.parquet' con {len(df_predecir)} filas.")

DataFrame de entrenamiento guardado en 'train.parquet' con 29652 filas.
DataFrame para predicción guardado en 'predecir.parquet' con 1870 filas.


In [116]:
df.shape

(31522, 280)

In [117]:
df.head(300).to_excel('head_producto.xlsx',sheet_name='hoja1',index=False)